# Customer Data Pre-processing

**Exercise 1: Read in the provided ACW Data using the CSV library.**

In [ ]:
#Import the CSV module for handling CSV files
import csv

**Exercise 2:Convert all flat structures into nested structures.**

In [ ]:
#Create empty lists to store the header information
header=[]

#Create an empty list to store the nested values 
nested_list=[]

# Open the CSV file with context manager for automatic closing
with open ("acw_user_data.csv", "r") as acw_file:
    acw_read=csv.DictReader(acw_file)
    acw_data=[line for line in acw_read]
    
# Extract header information from the first line
    for line_number, line in enumerate(acw_data):
            if line_number==0: 
                header.extend(line)     
                print (header)
                
 # A Function to convert flat data to nested structures   
    def convert_to_nested_structures(acw_data):
       
        """
        This function converts flat data from a CSV file to a list of dictionaries with nested structures.

        It recieves a list of dictionaries containing flat data from each line of the CSV file.

        It returns a list of dictionaries with unnested and nested structures like Vehicle, Address, and Credit Card.
        """
        for line in acw_data:
            try:
               #nest all the keys and values related to vehicle
                vehicle_data= {
                        "make":line['Vehicle Make'],
                        'model':line['Vehicle Model'],
                        'year':int(line['Vehicle Year']),
                        'type':line['Vehicle Type']
                }
                #nest all the address keys and values
                address_data={
                    'street':line['Address Street'],
                    'city':line['Address City'],
                    'postcode':line['Address Postcode']
                }
                #nest all the credit card keys and values
                card_data={
                    'start_date':line['Credit Card Start Date'],
                    'expiry_date':line['Credit Card Expiry Date'],
                    'number':int(line['Credit Card Number']),
                    "cvv" :int(line['Credit Card CVV']),
                    "iban":line["Bank IBAN"]
                }
                #nest the rest of the columns in the data
                unnested_columns={
                    'first_name':line['First Name'],
                    'last_name':line['Last Name'],
                    'age':int(line['Age (Years)']),
                    'sex':line["Sex"],
                    'retired':line["Retired"],
                    'marital_status':line['Marital Status'],
                    'dependants':line['Dependants'],
                    'salary':int(line['Yearly Salary (GBP)']),
                    'pension':int(line[ 'Yearly Pension (GBP)']),
                    'company':line['Employer Company'],
                    'commute_distance':float(line['Distance Commuted to Work (Km)'])
                }
                #append the nested columns to the 
                unnested_columns.update({ 
                            'Vehicle':vehicle_data,
                             'Credit card':card_data,
                             'Address': address_data
                })

                nested_list.append(unnested_columns)
                
            # Handle missing fields, data type issues, unexpected errors
            except TypeError as e:
                print(f"Error at line {line_number + 1}: {e} in data format.")
            except ValueError as e:
                print(f"Error at line {line_number + 1}: {e} in data format.")    
            except Exception as e:
                print(f"Unexpected error at line {line_number + 1}: {e}")    
        
        #return the list consiting of the nested and unnested columns 
        return(nested_list)
    
#Give the function a variable name for easy calling
nested_data=convert_to_nested_structures(acw_data)

# print the new data of nested and unnested structures
print(nested_data)

**Exercise 3: Find empty or whitespace entries in the "Dependants" column and print a list where all such error corrections take place.**

In [ ]:
def handle_empty_dependant(data):
    """
    This function finds and replaces empty or whitespace values in the Dependants column

    It recieves a list of dictionaries containing flat data from each line of the CSV file.

    It returns a list of the indexes with empty strings.
    """
    #initialise a list to store the index of empty or whitespace values
    empty_dependent_indexes=[]
    for index, line in enumerate(data):
        try:
            #Check for empty or whitespace values in the dependant column
            if line["dependants"] == "" or line["dependants"] == " ":
                empty_dependent_indexes.append(index)  # Append the index to the empty dependent list   
                
                #Replace all empty or white space entries with 0. 
                #Rationale: All customers have dependants which seems unlikely. 
                #The assumption is those with empty strings have no dependants
                line["dependants"] = 0
                
        #Check missing dependant column error        
        except KeyError:
            print(f"Error at index {index}: 'dependants' column not found.") 
            
    #returns the list of all the indexes    
    return(f"Problematic rows for dependants:, {empty_dependent_indexes}")
handle_empty_dependant(nested_data)  

**Exercise 4: Write all records to a processed.json file in the JSON data format.**

In [ ]:
#import JSON module to read and write JSON files
import json

# Open the processed JSON file in write mode with context manager for automatic closing
with open('processed.json', mode='w') as json_file:
    try:
        json.dump(nested_data, json_file) #write the nested data to the JSON file 
    
    #check for missing file error or any unexpected error
    except FileNotFoundError:
        print("The specified file was not found")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

**Exercise 5: Create two additional file outputs, retired.json and employed.json,
these should contain all retired customers and all employed customers.**

In [ ]:
#filters the data  for employed custmers by finding rows that have company names and cummuted distance. 
employed_data= [line for line in nested_data if line['company'] != 'N/A']
print(employed_data)

#Open the employed JSON file in write mode with context manager for automatic closing
with open('employed.json', mode='w') as employed_file:
    try:
        json.dump(employed_data, employed_file) #write the employed data to the JSON file 
    
    #check for missing file error or any unexpected error
    except FileNotFoundError:
        print("The specified file was not found")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
#Filters the data for retired customers by finding rows where retired is True
retired_data= [line for line in nested_data if line['retired'] == "True" ]
print(retired_data)

#Open the retired employed JSON file in write mode with context manager for automatic closing
with open('retired.json', mode='w') as retired_file:
    try:
        json.dump(retired_data, retired_file)  #write the retired data to the JSON file 
    
    #check for missing file error or any unexpected error
    except FileNotFoundError:
        print("The specified file was not found")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

**Exercise 6: Write a function to flag cards with more than 10 years validity and remove customers with those cards from the CSV data. write the removed cards to a removed_ccard JSON FILE** 

In [ ]:
#Import the datetime module to convert values to date format
from datetime import datetime

def flag_card(data_list):
    """
    This function flags cards with a validity period of more than 10 years.
    
    It recieves a list of dictionaries representing customer data.
    
    and returns True if the card has a validity period greater than 10 years, False otherwise.
    """
    for row in data_list:
        try:
            start_date=datetime.strptime(row['Credit Card Start Date'], '%m/%y').date() #convert the start_data values to date format
            expiry_date=datetime.strptime(row['Credit Card Expiry Date'], '%m/%y').date()#convert the expiry_date values to date format

            # Calculate the validity period of the card
            card_validity = (expiry_date.year-start_date.year)*12 +(expiry_date.month-start_date.month)
            
            #check if the validity of the card is more than 10 years (120 months)
            if card_validity > 120:    
                return True #returns true if the card validity is more than 10 years
            
        # Handle missing fields, data type issues, unexpected errors
        except KeyError as e:
            print(f"KeyError: {e} Required key not found in the row.")
        except ValueError as e:
            print(f"ValueError: {e} Error in value format.")
        except TypeError as e:
            print(f"TypeError: {e} Unexpected data type.")
        except Exception as e:
            print(f"Unexpected error: {e}") 
    
    #returns false if the card validity is less than 10 years
    return False

In [ ]:
def remove_card(data):
    """
    This function removes cards with a validity period of more than 10 years.

    It recieves a list of dictionaries representing customer data.

    It returns a list of dictionaries where the cards are more than 10 years
    """
    #initialize an empty list to store the records where the card validity is more than 10 years
    flagged_cards = []

    for row in data:
        try:
            if flag_card([row]): #calls the flag_card function for each row
                flagged_cards.append(row) #if True, the row is appended to the flagged_card list
            if not flagged_cards:
                print('No cards to remove')#If False, it prints No card to remove.
    
         # Handle missing fields, data type issues, unexpected errors
        except KeyError as e:
            print(f"KeyError: {e} Required key not found in the row.")
        except ValueError as e:
            print(f"ValueError: {e} Error in value format.")
        except TypeError as e:
            print(f"TypeError: {e} Unexpected data type.")
        except Exception as e:
            print(f"Unexpected error: {e}")

    #returns a list of all cusstomers with card validity of above 10 years   
    return flagged_cards

removed_cards = remove_card(acw_data) #call the function for the customer data
print(removed_cards)

In [ ]:
#Open the remove_ccard JSON file in write mode with context manager for automatic closing
with open('remove_ccard.json', mode='w') as rccard_file: 
    try:
        json.dump(removed_cards, rccard_file) #write the removed card data to the JSON file
    
    #check for missing file error or any unexpected error
    except FileNotFoundError:
        print("The specified file was not found")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

**Exercise 7: Create a new data attribute “Salary-Commute" by calculating the Salary that a customer earns, per mile of their commute. Write the new metric to a commute JSON file.**

In [ ]:
# Open the JSON file with context manager for automatic closing
with open('processed.json', mode='r') as json_file:
    try:
        processed_data= json.load(json_file)
    except FileNotFoundError:
        print("Error: The specified file was not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    def create_commute (data):
        """
        This function calculate salary per mile based on salary and commute distance.

        It recieves a list of dictionaries representing processed data.

        and returns a list of calculated salary per mile values.
        """
        
        salary_commute=[]
        for row in data:
            try:    
                #Extract salary and commute distance from the row
                yearly_salary=row['salary']
                commute=row['commute_distance']
                
                #Calculate salary per mile, handling zero commute distance
                if commute<=1:
                    salary_per_mile=yearly_salary
                else:
                    salary_per_mile=yearly_salary/commute
                    
                 #Round the result and append to the list
                salary_commute.append(round(salary_per_mile, 2))
                
            #Check error for cases where commute distance value is 0
            except ZeroDivisionError:
                print("You cannot divide by Zero")
                
        # Return a sorted list of calculated salary per mile values
        return sorted(salary_commute)
    
## Create a list of salary per mile values based on processed data    
commute_list=create_commute (processed_data)
print(commute_list)

In [ ]:
try:
    # Sort processed data based on the calculated salary per mile values
    processed_data_sorted = sorted(processed_data, key=lambda row: create_commute([row])[0])
except Exception as e:
    print(f"Error sorting data: {e}")

# Print the sorted processed data
print(processed_data_sorted)

In [ ]:
#Open the commute JSON file in write mode with context manager for automatic closing
with open('commute.json', mode='w') as commute_file:
    json.dump(processed_data_sorted, commute_file)#write the commute data to the JSON file

# Data Visualization

In [ ]:
# Import the pandas library as pd
import pandas as pd
#Import the seaborn library as sns
import seaborn as sns

In [ ]:
#Read the CSV file and create a dataframe
acw_df=pd.read_csv("acw_user_data.csv") 

**Exercise 1a: Calculate the Mean Salary**

In [ ]:
#Obtain the Data Series for Salary
salary=acw_df['Yearly Salary (GBP)']

mean_salary=salary.mean() #Calculate the average salary of the customers
print(f'The average salary of customers is: {mean_salary}') 

**Exercise 1b: Calculate the Mean Age**

In [ ]:
#Obtain the Data Series for Age
age=acw_df['Age (Years)']
mean_age=age.mean()
print(f'The average age of customers is: {mean_age}')

**Exercice 2a: Univariate plot of Age**

In [ ]:
#calculate how many bins would be required for a bin_width of 5 using range/bindwith.
bin_width=5
max_value=max(age)
min_value=min(age)
#calculate the range 
range_value=max_value-min_value
#calculate the bin
bin_value=round(range_value/bin_width)
print(bin_value) #15 bins would be required
age_plot=sns.displot(x=age, bins=15, label='Age')

**Exercise 2b: Univariate plot of Dependents, fixing data errors with seaborn**

In [ ]:
#Fill the Na values with 0 using .fillna()
dependants_filled=acw_df['Dependants'].fillna('0')
#Make the plot using the filled data series
dependant_plot=sns.displot(data=acw_df, x=dependants_filled, label='Dependants')

**Exercise 2c: Univariate plot Age conditioned on Marital Status**

In [ ]:
#Intesify the colour scheme to make the plot more visible
distinct_palette = sns.color_palette("bright")[:4] #isolate first four colours in the palette

#Obtain the data series for marital status
marital_status=acw_df['Marital Status']

#Plot the age column 
age_marital_plot=sns.displot(x=age, hue= marital_status, palette=distinct_palette, label='Age and Marital status')

**Exercise 3a: Mulivariate plot of Commuted distance against salary.**

In [ ]:
#create a scatter plot of commuted disatnce and Yearly salary
commute_salary_scatterplot=sns.scatterplot(data=acw_df, x='Distance Commuted to Work (Km)', y='Yearly Salary (GBP)', label='Distance comutted vs Salary')

**Exercise 3b: Multivariate plot of Age against Salary**

In [ ]:
age_salary_scatterplot=sns.scatterplot(data=acw_df, x='Age (Years)', y='Yearly Salary (GBP)', label='Age vs Salary')

**Exercise 3c: Multivariate plot of Age against Salary, conditioned by dependents**

In [ ]:
age_salary_dependants_scatterplot=sns.scatterplot(data=acw_df, x='Age (Years)', y='Yearly Salary (GBP)', hue=dependants_filled, label='Age vs Salary by no. of Dependants')

**Exercise 4: Save the plots**

In [ ]:
#Save all the plots to files
try:
    age_plot.savefig('./age_plot.png')
    dependant_plot.savefig('./dependant_plot.png')
    age_marital_plot.savefig('./age_marital_plot.png')
    commute_salary_scatterplot.get_figure().savefig('./commute_salary_scatterplot.png')
    age_salary_scatterplot.get_figure().savefig(('./age_salary_scatterplot.png'))
    age_salary_dependants_scatterplot.get_figure().savefig('./age_salary_dependents_scatterplot.png')
except Exception as e:
    print(f"Error saving plots: {e}")